In [1]:
# Dependencies and Setup
import pandas as pd

In [2]:
# Create the list of years/months that match individual dataset names
import_files = []

for x in range(2019,2021):
    for y in range(1,13):
        if (len(str(y)) == 1):
            month = '0' + str(y)
            import_files.append("../Resources/" + str(x) + month + "-citibike-tripdata.csv")
            import_files.append("../Resources/JC-" + str(x) + month + "-citibike-tripdata.csv")
        else:
            month = str(y)
            import_files.append("../Resources/" + str(x) + month + "-citibike-tripdata.csv")
            import_files.append("../Resources/JC-" + str(x) + month + "-citibike-tripdata.csv")
            
# Data format changes beginning Feb'21, so also append Jan'22 in old format
import_files.append("../Resources/202101-citibike-tripdata.csv")
import_files.append("../Resources/JC-202101-citibike-tripdata.csv")

In [3]:
# Loop through NY 2019-2021 CSV files and combine
# (Note: Downloaded files saved outside GitHub respository due to memory restrictions)
df_list = (pd.read_csv(file) for file in import_files)

# Concatenate all DataFrames
df = pd.concat(df_list, ignore_index=True)

In [5]:
df['usertype'] = df['usertype'].replace(to_replace='Subscriber', value='member')
df['usertype'] = df['usertype'].replace(to_replace='Customer', value='casual')
df['rideable_type'] = 'N/A'

In [4]:
# Create the list of years/months that match individual dataset names
import_files_2021 = []

for x in range(2021,2022):
    for y in range(2,13):
        if (len(str(y)) == 1):
            month = '0' + str(y)
            import_files_2021.append("../Resources/" + str(x) + month + "-citibike-tripdata.csv")
            import_files_2021.append("../Resources/JC-" + str(x) + month + "-citibike-tripdata.csv")
        else:
            month = str(y)
            import_files_2021.append("../Resources/" + str(x) + month + "-citibike-tripdata.csv")
            import_files_2021.append("../Resources/JC-" + str(x) + month + "-citibike-tripdata.csv")   

In [6]:
# Loop through NY 2019-2021 CSV files and combine
# (Note: Downloaded files saved outside GitHub respository due to memory restrictions)
df_list_2021 = (pd.read_csv(file) for file in import_files_2021)

# Concatenate all DataFrames
df_2021 = pd.concat(df_list_2021, ignore_index=True)
df_2021

/Users/coreylawson-enos/opt/anaconda3/envs/PythonData38/lib/python3.8/site-packages/pandas/core/reshape/concat.py:348: DtypeWarning: Columns (5,7) have mixed types.Specify dtype option on import or set low_memory=False.
  objs = list(objs)
/Users/coreylawson-enos/opt/anaconda3/envs/PythonData38/lib/python3.8/site-packages/pandas/core/reshape/concat.py:348: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  objs = list(objs)


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,BBA33D73DECE976F,docked_bike,2021-02-26 16:38:54,2021-02-26 16:44:37,E 84 St & Park Ave,7243.04,E 78 St & 2 Ave,7057.07,40.778626,-73.957720,40.772797,-73.955778,casual
1,B63D7AFF9AC5B6D4,docked_bike,2021-02-17 11:09:11,2021-02-17 11:26:47,Macon St & Nostrand Ave,4214.03,Bond St & Fulton St,4479.06,40.680983,-73.950047,40.689622,-73.983043,member
2,52B829195C469D99,docked_bike,2021-02-26 18:33:29,2021-02-26 19:05:41,Macon St & Nostrand Ave,4214.03,Lefferts Pl & Franklin Ave,4222.02,40.680983,-73.950047,40.680342,-73.955769,casual
3,19C84ECA2B468476,docked_bike,2021-02-26 12:48:35,2021-02-26 13:07:24,Macon St & Nostrand Ave,4214.03,Bond St & Fulton St,4479.06,40.680983,-73.950047,40.689622,-73.983043,member
4,C0DDB771E70D9DF5,docked_bike,2021-02-25 17:23:22,2021-02-25 17:28:20,Madison Ave & E 26 St,6131.12,W 37 St & 5 Ave,6398.06,40.742685,-73.986713,40.750380,-73.983390,member
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27198919,13FE513C650901A3,classic_bike,2021-12-16 17:17:46,2021-12-16 17:22:53,Grove St PATH,JC005,Hamilton Park,JC009,40.719586,-74.043117,40.727596,-74.044247,member
27198920,CA29051F38E67D30,classic_bike,2021-12-07 17:10:29,2021-12-07 17:15:02,Grove St PATH,JC005,Hamilton Park,JC009,40.719586,-74.043117,40.727596,-74.044247,member
27198921,A719DEA143005404,classic_bike,2021-12-18 09:50:14,2021-12-18 09:55:55,Grove St PATH,JC005,Hamilton Park,JC009,40.719586,-74.043117,40.727596,-74.044247,member
27198922,D39126975208B59D,classic_bike,2021-12-10 21:43:32,2021-12-10 21:49:04,Grove St PATH,JC005,Hamilton Park,JC009,40.719586,-74.043117,40.727596,-74.044247,member


In [9]:
# Delete me
df_2021['rideable_type'].unique()

array(['docked_bike', 'electric_bike', 'classic_bike'], dtype=object)

In [10]:
df_2021['started_at'] = pd.to_datetime(df_2021['started_at'])
df_2021['ended_at'] = pd.to_datetime(df_2021['ended_at'])

In [11]:
df_2021_clean = pd.DataFrame({
    'tripduration': (df_2021['ended_at'][0] - df_2021['started_at'][0]).total_seconds(),
    'starttime': df_2021['started_at'],
    'stoptime': df_2021['ended_at'],
    'start station id': df_2021['start_station_id'],
    'start station name': df_2021['start_station_name'],
    'start station latitude': df_2021['start_lat'],
    'start station longitude': df_2021['start_lng'],
    'end station id': df_2021['end_station_id'],
    'end station name': df_2021['end_station_name'],
    'end station latitude': df_2021['end_lat'],
    'end station longitude': df_2021['end_lng'],
    'bikeid': 'N/A',
    'usertype': df_2021['member_casual'],
    'birth year': 'N/A',
    'gender': 'N/A',
    'rideable_type': df_2021['rideable_type']
    })

In [12]:
df_2021_clean

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,rideable_type
0,343.0,2021-02-26 16:38:54,2021-02-26 16:44:37,7243.04,E 84 St & Park Ave,40.778626,-73.957720,7057.07,E 78 St & 2 Ave,40.772797,-73.955778,N/A,casual,N/A,N/A,docked_bike
1,343.0,2021-02-17 11:09:11,2021-02-17 11:26:47,4214.03,Macon St & Nostrand Ave,40.680983,-73.950047,4479.06,Bond St & Fulton St,40.689622,-73.983043,N/A,member,N/A,N/A,docked_bike
2,343.0,2021-02-26 18:33:29,2021-02-26 19:05:41,4214.03,Macon St & Nostrand Ave,40.680983,-73.950047,4222.02,Lefferts Pl & Franklin Ave,40.680342,-73.955769,N/A,casual,N/A,N/A,docked_bike
3,343.0,2021-02-26 12:48:35,2021-02-26 13:07:24,4214.03,Macon St & Nostrand Ave,40.680983,-73.950047,4479.06,Bond St & Fulton St,40.689622,-73.983043,N/A,member,N/A,N/A,docked_bike
4,343.0,2021-02-25 17:23:22,2021-02-25 17:28:20,6131.12,Madison Ave & E 26 St,40.742685,-73.986713,6398.06,W 37 St & 5 Ave,40.750380,-73.983390,N/A,member,N/A,N/A,docked_bike
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27198919,343.0,2021-12-16 17:17:46,2021-12-16 17:22:53,JC005,Grove St PATH,40.719586,-74.043117,JC009,Hamilton Park,40.727596,-74.044247,N/A,member,N/A,N/A,classic_bike
27198920,343.0,2021-12-07 17:10:29,2021-12-07 17:15:02,JC005,Grove St PATH,40.719586,-74.043117,JC009,Hamilton Park,40.727596,-74.044247,N/A,member,N/A,N/A,classic_bike
27198921,343.0,2021-12-18 09:50:14,2021-12-18 09:55:55,JC005,Grove St PATH,40.719586,-74.043117,JC009,Hamilton Park,40.727596,-74.044247,N/A,member,N/A,N/A,classic_bike
27198922,343.0,2021-12-10 21:43:32,2021-12-10 21:49:04,JC005,Grove St PATH,40.719586,-74.043117,JC009,Hamilton Park,40.727596,-74.044247,N/A,member,N/A,N/A,classic_bike


In [13]:
df = df.append(df_2021_clean, ignore_index = True)
len(df)

69106197

In [ ]:
df.dtypes

In [14]:
df['starttime'] = pd.to_datetime(df['starttime'])
df['stoptime'] = pd.to_datetime(df['stoptime'])
df['year_month'] = pd.to_datetime(df['starttime']).dt.to_period('M')

In [ ]:
df.dtypes

In [15]:
df_grouped = df.groupby([df['starttime'].dt.year.rename('year'), df['starttime'].dt.month_name().rename('month')])

In [ ]:
df_grouped.head()

In [16]:
total_rides = df_grouped['starttime'].count()
avg_duration = df_grouped['tripduration'].mean() / 60
total_member = df_grouped['usertype'].apply(lambda x: (x=='member').sum())
total_casual = df_grouped['usertype'].apply(lambda x: (x=='casual').sum())
total_classic = df_grouped['rideable_type'].apply(lambda x: (x=='classic_bike').sum())
total_electric = df_grouped['rideable_type'].apply(lambda x: (x=='electric_bike').sum())

In [18]:
time_summary = pd.DataFrame({
    'Total Rides': total_rides,
    'Average Ride (min)': avg_duration.round(0),
    'Member Rides': total_member,
    'Casual Rides': total_casual,
    'Classic Bike Rides': total_classic,
    'Electric Bike Rides': total_electric
    })
time_summary

Total Rides  Average Ride (min)  Member Rides  Casual Rides  \
year month                                                                    
2019 April          1799150                18.0       1566869        232281   
     August         2392935                17.0       1941681        451254   
     December        974938                14.0        896077         78861   
     February        962309                13.0        915520         46789   
     January         986963                13.0        946381         40582   
     July           2224810                17.0       1840550        384260   
     June           2164800                18.0       1786187        378613   
     March          1351566                15.0       1243831        107735   
     May            1960698                18.0       1662361        298337   
     November       1509505                14.0       1354619        154886   
     October        2134826                15.0       1835017        299809   
     September      2494144                17.0       2052349        441795   
2020 April           692030                27.0        523187        168843   
     August         2372781                23.0       1698242        674539   
     December       1100623                17.0        930021        170602   
     February       1169792                13.0       1076380         93412   
     January        1266616                13.0       1170255         96361   
     July           2144488                27.0       1554246        590242   
     June           1919194                28.0       1326513        592681   
     March          1086176                19.0        929260        156916   
     May            1512967                27.0       1025011        487956   
     November       1757979                19.0       1377915        380064   
     October        2292406                20.0       1732471        559935   
     September      2528607                23.0       1843796        684811   
2021 April          2091443                 6.0       1582357        509086   
     August         3166654                 6.0       2179088        987566   
     December       1796308                 6.0       1425774        370534   
     February        654864                 6.0        561396         93468   
     January        1106970                16.0        954817        152153   
     July           3167492                 6.0       2143228       1024264   
     June           3249523                 6.0       2252741        996782   
     March          1548217                 6.0       1215326        332891   
     May            2769208                 6.0       1937937        831271   
     November       2219345                 6.0       1724529        494816   
     October        3157997                 6.0       2316865        841132   
     September      3377873                 6.0       2344805       1033068   

                Classic Bike Rides  Electric Bike Rides  
year month                                               
2019 April                       0                    0  
     August                      0                    0  
     December                    0                    0  
     February                    0                    0  
     January                     0                    0  
     July                        0                    0  
     June                        0                    0  
     March                       0                    0  
     May                         0                    0  
     November                    0                    0  
     October                     0                    0  
     September                   0                    0  
2020 April                       0                    0  
     August                      0                    0  
     December                    0                    0  
     February              

In [20]:
time_summary.to_csv('Data/time_summary.csv')

In [24]:
df_by_ride = df[(df['rideable_type'] == 'classic_bike') | (df['rideable_type'] == 'electric_bike')]

In [25]:
df_grouped_user = df_by_ride.groupby([df['usertype']])

In [26]:
classic_rides = df_grouped_user['rideable_type'].apply(lambda x: (x=='classic_bike').sum())
electric_rides = df_grouped_user['rideable_type'].apply(lambda x: (x=='electric_bike').sum())

In [27]:
rides_summary = pd.DataFrame({
    'Classic': classic_rides,
    'Electric': electric_rides
    })
rides_summary

,Classic,Electric
usertype,,
casual,3336614,1862964
member,9548150,3851582


In [29]:
rides_summary.to_csv('Data/rides_summary.csv')

In [30]:
df_grouped_hour = df.groupby([df['starttime'].dt.year.rename('year'), df['starttime'].dt.hour.rename('Hour')])

In [31]:
count_per_hour = df_grouped_hour['starttime'].count()

In [33]:
count_per_hour.to_csv('Data/hour_summary.csv')

In [56]:
df = df[df['starttime'].notna()]
df = df[df['start station name'].notna()]
df = df[df['start station latitude'].notna()]
df = df[df['start station longitude'].notna()]

In [62]:
df_grouped_station = df.groupby([df['starttime'].dt.year.rename('year'), 
                                 df['starttime'].dt.month_name().rename('month'), 
                                 df['start station name'],
                                 df['start station latitude'],
                                 df['start station longitude']])

In [63]:
starts_by_station = df_grouped_station['starttime'].count()
starts_by_station

year  month      start station name            start station latitude  start station longitude
2019  April      1 Ave & E 110 St              40.792327               -73.938300                 1150
                 1 Ave & E 16 St               40.732219               -73.981656                 7213
                 1 Ave & E 18 St               40.733812               -73.980544                 4800
                 1 Ave & E 30 St               40.741444               -73.975361                 3928
                 1 Ave & E 44 St               40.750020               -73.969053                 3382
                                                                                                  ... 
2021  September  Wyckoff Ave & Gates Ave       40.699871               -73.911719                 1417
                 Wyckoff St & Nevins St        40.683426               -73.984275                  985
                 Wythe Ave & Metropolitan Ave  40.716887               -73.963198

In [59]:
# station_data = pd.DataFrame({
#     'Station': df_grouped_station['start station name'],
#     'Lat': df_grouped_station['start station latitude'],
#     'Lon': df_grouped_station['start station longitude'],
#     'Departures': starts_by_station
#     })
# station_data

Station  \
year month     start station name                                                                
2019 April     1 Ave & E 110 St              ((2019, April, 1 Ave & E 110 St), [1 Ave & E 1...   
               1 Ave & E 16 St               ((2019, April, 1 Ave & E 16 St), [1 Ave & E 16...   
               1 Ave & E 18 St               ((2019, April, 1 Ave & E 18 St), [1 Ave & E 18...   
               1 Ave & E 30 St               ((2019, April, 1 Ave & E 30 St), [1 Ave & E 30...   
               1 Ave & E 44 St               ((2019, April, 1 Ave & E 44 St), [1 Ave & E 44...   
...                                                                                        ...   
2021 September Wyckoff Av & Stanhope St      ((2021, September, Wyckoff Av & Stanhope St), ...   
               Wyckoff Ave & Gates Ave       ((2021, September, Wyckoff Ave & Gates Ave), [...   
               Wyckoff St & Nevins St        ((2021, September, Wyckoff St & Nevins St), [W...   
               Wythe Ave & Metropolitan Ave  ((2021, September, Wythe Ave & Metropolitan Av...   
               Yankee Ferry Terminal         ((2021, September, Yankee Ferry Terminal), [Ya...   

                                                                                           Lat  \
year month     start station name                                                                
2019 April     1 Ave & E 110 St              ((2019, April, 1 Ave & E 110 St), [40.7923272,...   
               1 Ave & E 16 St               ((2019, April, 1 Ave & E 16 St), [40.73221853,...   
               1 Ave & E 18 St               ((2019, April, 1 Ave & E 18 St), [40.733812191...   
               1 Ave & E 30 St               ((2019, April, 1 Ave & E 30 St), [40.74144387,...   
               1 Ave & E 44 St               ((2019, April, 1 Ave & E 44 St), [40.75001986,...   
...                                                                                        ...   
2021 September Wyckoff Av & Stanhope St      ((2021, September, Wyckoff Av & Stanhope St), ...   
               Wyckoff Ave & Gates Ave       ((2021, September, Wyckoff Ave & Gates Ave), [...   
               Wyckoff St & Nevins St        ((2021, September, Wyckoff St & Nevins St), [4...   
               Wythe Ave & Metropolitan Ave  ((2021, September, Wythe Ave & Metropolitan Av...   
               Yankee Ferry Terminal         ((2021, September, Yankee Ferry Terminal), [40...   

                                                                                           Lon  \
year month     start station name                                                                
2019 April     1 Ave & E 110 St              ((2019, April, 1 Ave & E 110 St), [-73.9383, -...   
               1 Ave & E 16 St               ((2019, April, 1 Ave & E 16 St), [-73.98165557...   
               1 Ave & E 18 St               ((2019, April, 1 Ave & E 18 St), [-73.98054420...   
               1 Ave & E 30 St               ((2019, April, 1 Ave & E 30 St), [-73.97536082...   
               1 Ave & E 44 St               ((2019, April, 1 Ave & E 44 St), [-73.96905301...   
...                                                                                        ...   
2021 September Wyckoff Av & Stanhope St      ((2021, September, Wyckoff Av & Stanhope St), ...   
               Wyckoff Ave & Gates Ave       ((2021, September, Wyckoff Ave & Gates Ave), [...   
               Wyckoff St & Nevins St        ((2021, September, Wyckoff St & Nevins St), [-...   
               Wythe Ave & Metropolitan Ave  ((2021, September, Wythe Ave & Metropolitan Av...   
               Yankee Ferry Terminal         ((2021, September, Yankee Ferry Terminal), [-7...   

                                             Departures  
year month     start station name                        
2019 April     1 Ave & E 110 St                    1150  
               1 Ave & E 16 St                     7213  
               1 Ave & E 18 St

In [65]:
starts_by_station.to_csv('Data/station_data.csv')

In [ ]:
df_2021_clean.head()

In [ ]:
len(df)

In [ ]:
data = {'Name':['Tom', 'Jack', 'Steve', 'Ricky'],'Age':[28,34,29,42]}
df = pd.DataFrame(data)

In [ ]:
# for file in glob.glob("../Resources/*.csv"):
#     f = zipfile.ZipFile(zip_file)
#     dfs = [pd.read_csv(zf.open(f), header=None, sep=";", encoding = 'unicode_escape') for f in zf.namelist()]
#     df = pd.concat(dfs,ignore_index=True)
#     print(df)
    
# path = "../Resources" # use your path
# all_files = glob.glob("../Resources/*.csv")

# li = []

# for filename in all_files:
#     df = pd.read_csv(filename, index_col=None, header=0)
#     li.append(df)

# frame = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
# Extract date and time fields for study
frame['starttime'] = pd.to_datetime(frame['starttime'])
frame['stoptime'] = pd.to_datetime(frame['stoptime'])
frame['year_month'] = pd.to_datetime(frame['starttime']).dt.to_period('M')
frame['duration_min'] = (frame['stoptime'] - frame['starttime']).dt.total_seconds() / 60
# frame['start_hour'] = frame['starttime'].hour

frame.sort_values('starttime')
frame.tail()

In [ ]:

# df = pd.concat(
#     map([pd.read_csv(name) for name in import_files], [import_files]), ignore_index=True)

# Create empty DataFrame for appending data
# df = pd.DataFrame()

# dataframe = pd.concat([pd.read_csv(name, compression='zip') for name in import_files])
# 

# for month in year_month:
#     import_file = "../Resources/" + month + "-citibike-tripdata.csv"
#     print(import_file)